In [ ]:
!pip install requests cachetools


In [2]:
# Configurations
TOGETHER_AI_API_KEY = "38936c79652e2c968f6e0259647de214db00afd3b87814d02e380bfc080bf04c"
TOGETHER_API_URL = "https://api.together.ai/llama3"
COIN_GECKO_API_URL = "https://api.coingecko.com/api/v3/simple/price"


In [3]:
import requests
from cachetools import TTLCache
import time

# Set up caching and rate limiting
crypto_cache = TTLCache(maxsize=10, ttl=300)  # Cache with a TTL of 5 minutes
LAST_API_CALL = 0
RATE_LIMIT = 2  # seconds

def rate_limited(func):
    def wrapper(*args, **kwargs):
        global LAST_API_CALL
        elapsed_time = time.time() - LAST_API_CALL
        if elapsed_time < RATE_LIMIT:
            time.sleep(RATE_LIMIT - elapsed_time)
        LAST_API_CALL = time.time()
        return func(*args, **kwargs)
    return wrapper

@rate_limited
def fetch_crypto_price(crypto="bitcoin"):
    if crypto in crypto_cache:
        return crypto_cache[crypto]

    try:
        response = requests.get(f"{COIN_GECKO_API_URL}?ids={crypto}&vs_currencies=usd")
        response.raise_for_status()
        data = response.json()
        price = data.get(crypto, {}).get("usd", None)
        if price:
            crypto_cache[crypto] = price  # Cache the result
            return price
        else:
            return "Price data not available."
    except requests.RequestException as e:
        return f"Error fetching price: {str(e)}"

def query_llama(prompt: str):
    headers = {
        "Authorization": f"Bearer {TOGETHER_AI_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {"prompt": prompt}

    try:
        response = requests.post(TOGETHER_API_URL, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return data.get("text", "Response unavailable.")
    except requests.RequestException as e:
        return f"Error with Together AI API: {str(e)}"


In [4]:
class AIAgent:
    def __init__(self):
        self.conversation_context = []

    def process_message(self, user_message):
        if "price" in user_message:
            crypto = "bitcoin"  # Default to Bitcoin
            if "ethereum" in user_message:
                crypto = "ethereum"
            price = fetch_crypto_price(crypto)
            response = f"The current price of {crypto.capitalize()} is ${price}."

        elif "translate" in user_message:
            lang_request = self.detect_language(user_message)
            response = f"Sure, but my responses will stay in English. What would you like to translate?"

        else:
            # Default handling for Together AI conversation
            prompt = self.build_prompt(user_message)
            response = query_llama(prompt)

        # Update the conversation context
        self.conversation_context.append({"user": user_message, "agent": response})
        return response

    def build_prompt(self, user_message):
        context_text = "\n".join([f"User: {msg['user']}\nAgent: {msg['agent']}" for msg in self.conversation_context[-5:]])
        return f"{context_text}\nUser: {user_message}\nAgent:"

    def detect_language(self, text):
        # Simplified language detection stub
        return "en" if "english" in text.lower() else "unknown"


In [5]:
# Initialize the agent
agent = AIAgent()

# Simulate user queries
print(agent.process_message("What is the Bitcoin price?"))
print(agent.process_message("Can you translate this to French?"))


The current price of Bitcoin is $69391.
Sure, but my responses will stay in English. What would you like to translate?
